In [1]:
import os
import pandas as pd
import numpy as np 
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from datetime import datetime, timedelta
import logging


In [2]:
load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

SOURCE_TABLE_A = "DS_BKC_Infineon_TMP-NVD-01_BY_LOT_Daily_6AM"
SOURCE_TABLE_B = "DS_BKC_MSP_Yield_Lot"
TARGET_TABLE = "DS_BKC_PACK_MSP"

In [3]:
def get_current_week_range_sat_fri():
    """
    Calculates the start (Saturday) and end date (Friday) of the week
    containing the current day.
    """
    today = datetime.now().date()
    # Calculate days to subtract to get to the *previous* Saturday
    # today.weekday(): Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6
    # We want Saturday (5) to be day 0 of our target week.
    # Offset: Sat=0, Sun=1, Mon=2, Tue=3, Wed=4, Thu=5, Fri=6
    days_since_last_saturday = (today.weekday() + 2) % 7
    start_of_week = today - timedelta(days=days_since_last_saturday) # Saturday
    end_of_week = start_of_week + timedelta(days=6) # Friday
    logging.info(f"Current Sat-Fri week range: {start_of_week} to {end_of_week}")
    return start_of_week, end_of_week

In [ ]:
try:
    # Construct the database URL
    db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    
    # Create the engine
    engine = create_engine(db_url)
    
    # Connect to the database
    with engine.connect() as connection:
        logging.info("Successfully connected to the PostgreSQL database.")

        print('Successfully connected to the PostgreSQL database.')
        # Execute a simple query to test the connection
        connection.execute(text("SELECT 1"))
except Exception as e:
    logging.error(f"Database connection failed: {e}")